In [1]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# custom_react_agent.py
from typing import Annotated, List, TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# ---------------------------
# 1. Define Tools
# ---------------------------
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

@tool
def divide(a: int, b: int) -> float:
    """Divide a by b."""
    if b == 0:
        return "Error: Division by zero"
    return a / b

TOOLS = {t.name: t for t in [multiply, divide]}

# ---------------------------
# 2. Define Agent State
# ---------------------------
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], "The conversation history"]

# ---------------------------
# 3. LLM Node
# ---------------------------
llm = ChatOpenAI(model="gpt-4o-mini")

def call_model(state: AgentState):
    """LLM decides whether to call a tool or give final answer."""
    response = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}

# ---------------------------
# 4. Tool Node
# ---------------------------
def call_tool(state: AgentState):
    last_message = state["messages"][-1]
    if not hasattr(last_message, "tool_calls") or not last_message.tool_calls:
        # No tool call requested
        return {"messages": state["messages"]}
    
    tool_msgs = []
    for call in last_message.tool_calls:
        tool_name = call["name"]
        tool_input = call["args"]
        tool = TOOLS.get(tool_name)
        if tool:
            result = tool.invoke(tool_input)
            tool_msgs.append(ToolMessage(content=str(result), tool_call_id=call["id"]))
    return {"messages": state["messages"] + tool_msgs}

# ---------------------------
# 5. Conditional Edge
# ---------------------------
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and getattr(last_message, "tool_calls", None):
        return "tool"
    return END

# ---------------------------
# 6. Build Graph
# ---------------------------
workflow = StateGraph(AgentState)

workflow.add_node("model", call_model)
workflow.add_node("tool", call_tool)

workflow.add_edge(START, "model")
workflow.add_conditional_edges("model", should_continue, {"tool": "tool", END: END})
workflow.add_edge("tool", "model")

graph = workflow.compile()

# ---------------------------
# 7. Run Example
# ---------------------------
if __name__ == "__main__":
    inputs = {"messages": [HumanMessage(content="What is (5 * 6) / 3 ?")]}
    result = graph.invoke(inputs)
    print(result["messages"][-1].content)
